# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
train_df = pd.read_csv(url, index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
train_df.shape

(6000, 61)

In [3]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular']

In [4]:
y.mean()

0.5

In [5]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [6]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1)
models = {'lr': LogisticRegressionCV(),
          'dt': DecisionTreeClassifier()}
for model in models.keys():
    models[model].fit(X_train, y_train)
# predict test for each model
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [7]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import numpy as np
from sklearn.metrics import f1_score
for model in models.keys():
    print(model)
    print('F1 Score: ', f1_score(y_pred[model], y_test))
    print('Accurancy: ', metrics.accuracy_score(y_pred[model], y_test))

lr
F1 Score:  0.5965156794425087
Accurancy:  0.614
dt
F1 Score:  0.5402144772117962
Accurancy:  0.5426666666666666


In [8]:
np.sqrt(mean_squared_error(y_pred.mean(axis=1), y_test))

0.5673035636529471

Por F1-score y el Acc se evidencia que es mejor el modelo de Logit 

# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [9]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingClassifier
models = {'log': LogisticRegression(),
          'dt_1': DecisionTreeClassifier(max_depth=None),
          'dt_2': DecisionTreeClassifier(max_depth=2)}
y_pred = pd.DataFrame (columns =models.keys(),index= y_test.index )
for i, model  in models.items():
    bagreg = BaggingClassifier(model, n_estimators=100,bootstrap=True, oob_score=True, random_state=1,n_jobs=-1)
    bagreg.fit(X_train, y_train)
    y_pred[i] = bagreg.predict(X_test)
y_pred.head()

,log,dt_1,dt_2
1483,1,1,1
2185,1,1,1
2520,1,1,1
3721,1,1,1
3727,0,0,0


# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [10]:
for model in models.keys():
    print(model)
    print('F1 Score: ', f1_score(y_pred[model], y_test))
    print('Accurancy: ', metrics.accuracy_score(y_pred[model], y_test))
print('Mean: ')
print('f1 score: ',metrics.f1_score(np.round(y_pred.mean(axis=1)).astype(int), y_test), 'Acc: ',metrics.accuracy_score(np.round(y_pred.mean(axis=1)).astype(int), y_test))

log
F1 Score:  0.6015352407536636
Accurancy:  0.6193333333333333
dt_1
F1 Score:  0.635752688172043
Accurancy:  0.6386666666666667
dt_2
F1 Score:  0.6434782608695653
Accurancy:  0.6446666666666667
Mean: 
f1 score:  0.6365475387727579 Acc:  0.6406666666666667


# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [11]:
th=[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
models = {'log': LogisticRegression(),
          'dt_1': DecisionTreeClassifier(max_depth=None),
          'dt_2': DecisionTreeClassifier(max_depth=2)}
f1_t = []
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for i in th:
    for model in models.keys():
        bagreg = BaggingClassifier(models[model], n_estimators=100,bootstrap=True, oob_score=True, random_state=1,n_jobs=-1)
        bagreg.fit(X_train, y_train)
        y_pred[model] = bagreg.predict_proba(X_test)
        y_pred[model] = y_pred[model].apply(lambda x: 1 if x>=i else 0)
        f1_t.append([i,f1_score(y_test,y_pred[model]),metrics.accuracy_score(y_pred[model], y_test),model])
f1_t=pd.DataFrame(f1_t, columns=['Threshold','F1 Score','Acc','Model'])

In [12]:
flog = f1_t[f1_t.Model=='log']
fdt1 = f1_t[f1_t.Model=='dt_1']
fdt2 = f1_t[f1_t.Model=='dt_2']
print('Best F1 Score in log:')
print(flog.loc[flog['F1 Score'].idxmax()])
print()
print('Best F1 Score in dt_1:')
print(fdt1.loc[fdt1['F1 Score'].idxmax()])
print()
print('Best F1 Score in dt_2:')
print(fdt2.loc[fdt2['F1 Score'].idxmax()])

Best F1 Score in log:
Threshold         0.1
F1 Score     0.667557
Acc             0.502
Model             log
Name: 27, dtype: object

Best F1 Score in dt_1:
Threshold         0.1
F1 Score     0.667851
Acc          0.501333
Model            dt_1
Name: 28, dtype: object

Best F1 Score in dt_2:
Threshold         0.2
F1 Score     0.670213
Acc             0.504
Model            dt_2
Name: 26, dtype: object


Sin modificar el Threshold pasa lo siguiente:

In [13]:
th=[0.5]
models = {'log': LogisticRegression(),
          'dt_1': DecisionTreeClassifier(max_depth=None),
          'dt_2': DecisionTreeClassifier(max_depth=2)}
f1_t = []
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for i in th:
    for model in models.keys():
        bagreg = BaggingClassifier(models[model], n_estimators=100,bootstrap=True, oob_score=True, random_state=1,n_jobs=-1)
        bagreg.fit(X_train, y_train)
        y_pred[model] = bagreg.predict_proba(X_test)
        y_pred[model] = y_pred[model].apply(lambda x: 1 if x>=i else 0)
        f1_t.append([i,f1_score(y_test,y_pred[model]),metrics.accuracy_score(y_pred[model], y_test),model])
f1_t=pd.DataFrame(f1_t, columns=['Threshold','F1 Score','Acc','Model'])
print()
print('Best F1 Score:')
print(f1_t.loc[f1_t['F1 Score'].idxmax()])
print()


Best F1 Score:
Threshold         0.5
F1 Score     0.411653
Acc          0.380667
Model             log
Name: 0, dtype: object



# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [14]:
models = {'log': LogisticRegression(),
          'dt_1': DecisionTreeClassifier(max_depth=None),
          'dt_2': DecisionTreeClassifier(max_depth=2)}
f1_t = []
mod2={}
for model in models.keys():
    mod2[model] = BaggingClassifier(models[model], n_estimators=100,bootstrap=True, oob_score=True, random_state=1,n_jobs=-1)
    mod2[model].fit(X_train, y_train)
    y_pred[model] = mod2[model].predict(X_test)
    f1_t.append([metrics.f1_score(y_pred[model], y_test), metrics.accuracy_score(y_pred[model], y_test),model])
f1_t=pd.DataFrame(f1_t, columns=['F1 Score','Acc','Model'])

In [15]:
f1=[]
for model in mod2.keys():
    errors = np.zeros(mod2[model].n_estimators)
    y_pred_all_ = np.zeros((X_test.shape[0], mod2[model].n_estimators))
    for i in range(mod2[model].n_estimators):
        oob_sample = ~mod2[model].estimators_samples_[i]
        y_pred_ = mod2[model].estimators_[i].predict(X_train.values[oob_sample])
        errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
        y_pred_all_[:, i] = mod2[model].estimators_[i].predict(X_test)
    alpha = (1 - errors) / (1 - errors).sum()
    y_pred[model] = (np.sum(y_pred_all_ * alpha, axis=1) >= 0.5).astype(np.int)
    f1.append([metrics.f1_score(y_pred[model], y_test), metrics.accuracy_score(y_pred[model], y_test),model])

In [16]:
f1=pd.DataFrame(f1, columns=['F1 Score','Acc','Model'])
print()
print('Best F1 Score:')
print(f1.loc[f1['F1 Score'].idxmax()])
print()


Best F1 Score:
F1 Score    0.643955
Acc         0.644667
Model           dt_2
Name: 2, dtype: object



# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [17]:
f2=[]
th=[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
for j in th:
    for model in mod2.keys():
        errors = np.zeros(mod2[model].n_estimators)
        y_pred_all_ = np.zeros((X_test.shape[0], mod2[model].n_estimators))
        for i in range(mod2[model].n_estimators):
            oob_sample = ~mod2[model].estimators_samples_[i]
            y_pred_ = mod2[model].estimators_[i].predict(X_train.values[oob_sample])
            errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
            y_pred_all_[:, i] = mod2[model].estimators_[i].predict(X_test)
        alpha = (1 - errors) / (1 - errors).sum()
        y_pred[model] = (np.sum(y_pred_all_ * alpha, axis=1) >= j).astype(np.int)
        f2.append([j, metrics.f1_score(y_pred[model], y_test), metrics.accuracy_score(y_pred[model], y_test),model])
f2=pd.DataFrame(f2, columns=['Threshold','F1 Score','Acc','Model'])
print()
print('Best F1 Score:')
print(f2.loc[f2['F1 Score'].idxmax()])
print()


Best F1 Score:
Threshold         0.3
F1 Score     0.697148
Acc          0.589333
Model            dt_1
Name: 22, dtype: object



# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score

In [18]:
ff=[]
for model in mod2.keys():
    y_pred_all_ = np.zeros((X_test.shape[0], mod2[model].n_estimators))
    X_train_3 = np.zeros((X_train.shape[0], mod2[model].n_estimators))
    for i in range(mod2[model].n_estimators):
        X_train_3[:, i] = mod2[model].estimators_[i].predict(X_train)
        y_pred_all_[:, i] = mod2[model].estimators_[i].predict(X_test)
    lr = LogisticRegressionCV(cv=5)
    lr.fit(X_train_3, y_train)
    y_pred[model] = lr.predict(y_pred_all_)
    ff.append([model, metrics.f1_score(y_pred[model], y_test), metrics.accuracy_score(y_pred[model], y_test)])
ff=pd.DataFrame(ff, columns=['Model','F1 Score','Acc'])
print()
print('Best F1 Score:')
print(ff.loc[ff['F1 Score'].idxmax()])
print()


Best F1 Score:
Model           dt_2
F1 Score    0.634048
Acc            0.636
Name: 2, dtype: object

